# Find a Legend Challenge - Submission Notebook

Competition:  https://xeek.ai/challenges/extract-crossplot-markers <br>
Repository: https://github.com/REDA-solutions/AdvancedCV-competition

In [1]:
team_name = 'Team Barenstark' 
model_name = 'best.pt'

## Imports

In [2]:
import os
import platform
import sys
import numpy as np
import pandas as pd
from glob import glob
import torch

from models_ocr.preprocessing.preprocessor import Preprocessor # import of our class --> see repository
from models_ocr.pytesseract_model import PytesseractModel # import of our class --> see repository

C:\Users\ti-he\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Description

![Model Architecture](misc/model_architecture.png)

## Submission inference pipeline

In [3]:
from time import perf_counter

In [4]:
TEST_DATA_ROOT = "raw_data/helvetios_challenge_dataset_test"
TEST_IMAGE_DATASET_PATH = f"{TEST_DATA_ROOT}/images"
TEST_LABELS_DATA_PATH = f"{TEST_DATA_ROOT}/labels"
TEST_INFERENCE_RESULTS_PATH = f"results"

In [5]:
def run_inference_pipeline(TEST_IMAGE_DATASET_PATH, TEST_INFERENCE_RESULTS_PATH):
   
   print(f"* OS                          : {platform.system()}, {platform.release()}")
   python_version = str(sys.version).replace('\n', ' ')
   print(f"* Python version              : {python_version}")

   os.makedirs(TEST_INFERENCE_RESULTS_PATH, exist_ok=True)
         
   ts_start = perf_counter()
   
   model = torch.hub.load('../yolov5/', 'custom', path='models_detection/best.pt', source='local')
   preprocessor = Preprocessor(deskew=True)
   tesseract = PytesseractModel(preprocessor=preprocessor, confidence=15)

   imgs = TEST_IMAGE_DATASET_PATH + r'/*.png'  
   imgs = list(glob(imgs))[:5]
   
   results = []
   sample_names = []

   for img in imgs:
      legend = model(img)
      legend = legend.crop()
      if len(legend) == 0:
         results.append(np.nan)
      else:
         legend = legend[0]['im']
         prediction = tesseract.predict(legend)
         prediction_str = "["
         for word in prediction: prediction_str += f"'{word}' "
         prediction_str = prediction_str.strip()
         prediction_str += "]"
         results.append(prediction_str)
      sample_names.append(img.split("\\")[1])
   
   ts_after_test = perf_counter()
   
   print(f"Inference time: {ts_after_test-ts_start:.2f} sec.")
   
   inference_results = {'sample_name': sample_names,
                        'legend': results}
   inference_results_df = pd.DataFrame(inference_results)
      
   inference_results_df.to_csv(f"{TEST_INFERENCE_RESULTS_PATH}/{team_name}_{model_name}_results.csv", index = False)
   
   print(inference_results_df)

   print(f"The submission file   : {TEST_INFERENCE_RESULTS_PATH}/{team_name}_{model_name}_results.csv")


In [6]:
run_inference_pipeline(TEST_IMAGE_DATASET_PATH, TEST_INFERENCE_RESULTS_PATH)

* OS                          : Windows, 10
* Python version              : 3.10.8 (tags/v3.10.8:aaaf517, Oct 11 2022, 16:50:30) [MSC v.1933 64 bit (AMD64)]


YOLOv5  v7.0-9-gf9ca365 Python-3.10.8 torch-1.13.0+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
Saved 1 image to runs\detect\exp1279
Saved results to runs\detect\exp1279

Saved 1 image to runs\detect\exp1280
Saved results to runs\detect\exp1280

Saved 1 image to runs\detect\exp1281
Saved results to runs\detect\exp1281

Saved 1 image to runs\detect\exp1282
Saved results to runs\detect\exp1282

Saved 1 image to runs\detect\exp1283
Saved results to runs\detect\exp1283



Inference time: 5.15 sec.
                sample_name                                 legend
0  20220915195651573677.png  ['drt' 'qn' '©' 'deo' 'RE' '@' 'ber']
1  20220915195652176684.png                                    NaN
2  20220915195652713648.png                      ['Mobility' 'ae']
3  20220915195653681519.png                                ['exe']
4  20220915195654121931.png                                    NaN
The submission file   : results/Team Barenstark_best.pt_results.csv
